In [1]:
import os

import regex as re
import pandas as pd

In [2]:
from collections import Counter

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package punkt to /home/varsey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/varsey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /home/varsey/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [4]:
from rapidfuzz import process as proc
from rapidfuzz import fuzz

In [5]:
from tqdm import tqdm

In [14]:
from stemmer_aze.stemmer import Stemmer

st = Stemmer()

FileNotFoundError: [Errno 2] No such file or directory: 'assets/words.txt'

In [16]:
data_folder = '/'.join(os.getcwd().split('/')[:-1])
data_folder

'/home/varsey/PycharmProjects/drl-case'

In [17]:
df_incoming = pd.read_csv(f'{data_folder}/data/incoming.csv')
df_outcoming = pd.read_csv(f'{data_folder}/data/outcoming.csv')

In [18]:
df_incoming.sample(3)

,Unnamed: 0,#,Product or Service Name,Unit,Quantity/Volume
4663,824904,824905,LAYS RIDGED PENDIR ILE SOGAN 150 GR 1 x 18,EDED,5
4328,635908,635909,TADIM KABAK CEKIRDEK 180 GR (PUMPKIN SEEDS) 1X12,EDED,12
955,768041,768042,SEMEYNIY BUDJ MAL ETI KONS BOQATR 325QR 1X24,EDED,12


In [19]:
df_outcoming.sample(3)

,Unnamed: 0,#,Product or Service Name,Unit,Quantity/Volume
3020,522195,522196,SEVIMLI DAD GATIG 0.5% 8 KG 1X1,EDED,3
4075,415761,415762,ULKER PENDIR LABNE 200 GR 1X10,EDED,10
1705,439996,439997,KIRLANGIC MISIR YAGI 1 LT PET 1X12,ADET.LT,12


In [20]:
df_outcoming['unit'] = df_incoming['Unit'].str.lower()
df_incoming['unit'] = df_incoming['Unit'].str.lower()

In [21]:
df_outcoming['unit'].value_counts()

unit
eded       4263
adet.lt     315
ədəd        166
adet.ml     128
adet.kg      75
adet.cl      41
litr          5
kq            3
kg            2
baql          1
amp           1
Name: count, dtype: int64

In [22]:
df_incoming['unit'].value_counts()

unit
eded       4263
adet.lt     315
ədəd        166
adet.ml     128
adet.kg      75
adet.cl      41
litr          5
kq            3
kg            2
baql          1
amp           1
Name: count, dtype: int64

In [23]:
Counter(" ".join(df_incoming['Product or Service Name']).split()).most_common(50)

[('GR', 3353),
 ('1', 949),
 ('1X12', 872),
 ('SEVIMLI', 828),
 ('DAD', 828),
 ('PINAR', 521),
 ('1x12', 500),
 ('x', 489),
 ('LAYS', 466),
 ('200', 448),
 ('180', 397),
 ('PRESIDENT', 334),
 ('LT', 300),
 ('KESMIK', 279),
 ('QR', 278),
 ('150', 267),
 ('90', 250),
 ('500', 241),
 ('1X24', 239),
 ('XAMA', 236),
 ('X', 218),
 ('1x1', 200),
 ('SUXARI', 195),
 ('50', 195),
 ('KG', 195),
 ('ML', 194),
 ('ULKER', 181),
 ('BEBETO', 174),
 ('18', 163),
 ('LABNE', 158),
 ('SUD', 156),
 ('ELVAN', 156),
 ('PENDIR', 156),
 ('1X15', 155),
 ('MILK', 154),
 ('KEFIR', 153),
 ('25', 153),
 ('DOKTOR', 150),
 ('20%', 148),
 ('TADIM', 147),
 ('1X8', 145),
 ('0%', 145),
 ('PEYNIR', 145),
 ('40', 144),
 ('24', 143),
 ('250', 140),
 ('900', 138),
 ('SUPERFRESH', 138),
 ('GATIG', 133),
 ('PAPRIKA', 132)]

In [24]:
flt_chars = [
    ')', '(', ',', '!', '', ':', '|', '>', '<', '[', ']', '...', './', '/\n', '\xa0',
    '«', '»', '_', '?', '~', '=', '\\', ';', '\n', '``', "''", '✅', '/n', '/', '..', '`', "'", '-',
    '✔', '⇒', '"', '$', '%', '&', "'", ',', '^', '`', '{', '}', "('",
]

In [25]:
stop_words = nltk.corpus.stopwords.words('russian') + nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('azerbaijani')
exclude = [x for x in set(stop_words + flt_chars) if len(x) > 0]
exclude.remove('up')
exclude.remove('s')

In [26]:
def process_item(x: str):
    for char in flt_chars:
        x = x.replace(char, ' ').lower()
    item = ' '.join(re.split(r'(\d+)', x))
    while item.count(2 * " ") > 0:
        item = item.replace(2 * " ", " ")
    return ' '.join(st.stem_words([x for x in item.split()]))

In [27]:
def simple_process_item(x: str):
    x = x.lower()
    # for char in flt_chars:
    #     x = x.replace(char, ' ')
    # print(x)
    x = (x.replace('*', ' x ').replace('-', ' ').replace("'", ' ').replace("(", ' ').replace(")", ' ').replace("%", 'pct').replace('"', ' ')
         .replace('<', ' ').replace('>', ' ')).replace(",", '.').replace("/", '.').replace(" s ", 's ')
    item = ' '.join(re.split(r'(\d+)', x))
    while item.count(2 * " ") > 0:
        item = item.replace(2 * " ", " ")
    item = item.replace(' .', '.').replace('. ', '.')
    return ' '.join([x for x in item.split() if x not in exclude])

In [28]:
exclude

['ours',
 'o',
 'иногда',
 'won',
 'нее',
 'again',
 'against',
 'dedi',
 'herself',
 'shan',
 "weren't",
 'только',
 'above',
 'лучше',
 'now',
 'из',
 'ж',
 'после',
 'did',
 'elə',
 'mən',
 'yourselves',
 "don't",
 'какой',
 'bilər',
 "she's",
 '}',
 'wasn',
 'off',
 'on',
 'üçün',
 ']',
 'xeyr',
 't',
 'haqqında',
 'each',
 'etmə',
 'от',
 "couldn't",
 'weren',
 'a',
 'sizin',
 'amma',
 'from',
 'will',
 'bax',
 'through',
 'вас',
 'ad',
 'çox',
 'было',
 'mənə',
 'artıq',
 'nor',
 'biri',
 'do',
 'ə',
 'mightn',
 'qırx',
 'siz',
 'have',
 'ma',
 'only',
 '/',
 '[',
 "isn't",
 'an',
 'что',
 'onu',
 'few',
 'there',
 'itself',
 "wouldn't",
 'ha',
 '(',
 ':',
 'yəni',
 'разве',
 ',',
 '\\',
 'no',
 'are',
 "haven't",
 'всего',
 'где',
 'будет',
 'ya',
 'вам',
 'ain',
 'bunu',
 'еще',
 'sənə',
 '_',
 'эти',
 'она',
 'over',
 'my',
 'we',
 'бы',
 'почти',
 'впрочем',
 'indi',
 'əlbəttə',
 'öz',
 './',
 'dörd',
 'onların ',
 'az',
 'ну',
 'ay',
 'who',
 'cü',
 'этого',
 'меня',
 'bunla

In [29]:
# exclude

In [30]:
text = df_outcoming.sample(1)['Product or Service Name'].values[0]
print(text)
simple_process_item(text)

LAYS RIDGED CHILI


'lays ridged chili'

In [31]:
text = df_incoming.sample(1)['Product or Service Name'].values[0]
print(text)
simple_process_item(text)

SEVIMLI DAD GATIG 1,5% 900 GR 1X6


'sevimli dad gatig 1.5 pct 900 gr 1 x 6'

In [32]:
df_outcoming['items'] = df_outcoming['Product or Service Name'].apply(lambda x: simple_process_item(x))
df_incoming['items'] = df_incoming['Product or Service Name'].apply(lambda x: simple_process_item(x))

In [33]:
Counter(" ".join(df_outcoming['items']).split()).most_common(150)

[('1', 5378),
 ('x', 5072),
 ('gr', 3748),
 ('12', 1452),
 ('pct', 1055),
 ('sevimli', 771),
 ('dad', 771),
 ('24', 637),
 ('pinar', 590),
 ('lays', 488),
 ('200', 480),
 ('6', 424),
 ('20', 397),
 ('180', 382),
 ('lt', 332),
 ('president', 324),
 ('500', 290),
 ('150', 286),
 ('qr', 267),
 ('90', 266),
 ('10', 256),
 ('15', 256),
 ('30', 256),
 ('suxari', 250),
 ('kesmik', 249),
 ('18', 247),
 ('ml', 233),
 ('xama', 228),
 ('25', 218),
 ('ulker', 199),
 ('40', 191),
 ('8', 191),
 ('16', 174),
 ('tadim', 172),
 ('50', 171),
 ('kg', 169),
 ('250', 165),
 ('labne', 157),
 ('bebeto', 156),
 ('0', 154),
 ('pendir', 152),
 ('60', 147),
 ('peynir', 145),
 ('300', 144),
 ('100', 142),
 ('kefir', 142),
 ('pet', 142),
 ('130', 138),
 ('4', 137),
 ('yagi', 136),
 ('9', 133),
 ('27', 133),
 ('doritos', 132),
 ('2.5', 131),
 ('sud', 130),
 ('paprika', 129),
 ('superfresh', 128),
 ('puding', 127),
 ('elvan', 127),
 ('0.5', 126),
 ('milk', 118),
 ('ridged', 115),
 ('sir', 115),
 ('pendiri', 115),
 (

In [34]:
Counter(" ".join(df_incoming['items']).split()).most_common(150)

[('1', 5423),
 ('x', 5071),
 ('gr', 3706),
 ('12', 1519),
 ('pct', 1133),
 ('sevimli', 828),
 ('dad', 828),
 ('24', 588),
 ('pinar', 521),
 ('200', 510),
 ('lays', 466),
 ('20', 414),
 ('6', 413),
 ('180', 408),
 ('president', 334),
 ('lt', 323),
 ('qr', 303),
 ('kesmik', 279),
 ('150', 270),
 ('30', 254),
 ('90', 254),
 ('15', 251),
 ('500', 248),
 ('25', 243),
 ('xama', 236),
 ('18', 223),
 ('kg', 218),
 ('ml', 212),
 ('10', 209),
 ('50', 207),
 ('suxari', 195),
 ('8', 190),
 ('ulker', 181),
 ('40', 178),
 ('16', 176),
 ('bebeto', 174),
 ('9', 159),
 ('labne', 158),
 ('0.5', 158),
 ('sud', 156),
 ('elvan', 156),
 ('pendir', 156),
 ('milk', 154),
 ('kefir', 153),
 ('250', 152),
 ('60', 151),
 ('doktor', 150),
 ('0', 149),
 ('tadim', 147),
 ('peynir', 145),
 ('900', 141),
 ('4', 140),
 ('superfresh', 138),
 ('yagli', 138),
 ('100', 133),
 ('gatig', 133),
 ('27', 132),
 ('paprika', 132),
 ('doritos', 130),
 ('300', 130),
 ('pet', 128),
 ('2.5', 126),
 ('yagi', 121),
 ('130', 119),
 ('40

In [35]:
def count_digit_share(x: str):
    counter = 0
    for char in x:
        if char.isdigit():
            counter += 1
    return counter/len(x.replace(' ', ''))

In [36]:
count_digit_share('7 up 1 lt x 12')

0.4444444444444444

In [37]:
def count_common_words(str1, str2):
    set1 = set([x for x in str1.lower().split() if len(x) > 3 and not x.isdigit()])
    set2 = set([x for x in str2.lower().split() if len(x) > 3 and not x.isdigit()])
    # print(set1.intersection(set2))
    # return len(set1.intersection(set2)), (len(set1) + len(set2))/2, len(set1.intersection(set2))/((len(set1) + len(set2))/2)
    if ((len(set1) + len(set2))/2) == 0:
        return 0
    return len(set1.intersection(set2))/((len(set1) + len(set2))/2)

In [38]:
def count_common_digits(str1, str2):
    set1 = set([x for x in str1.lower().split() if x.isdigit()])
    set2 = set([x for x in str2.lower().split() if x.isdigit()])
    # print(set1.intersection(set2))
    # return len(set1.intersection(set2)), (len(set1) + len(set2))/2, len(set1.intersection(set2))/((len(set1) + len(set2))/2)
    if ((len(set1) + len(set2))/2) == 0:
        return 0
    return len(set1.intersection(set2))/((len(set1) + len(set2))/2)

In [39]:
subquery = df_incoming.sample(1)['items'].values[0]
print(subquery)
answers = proc.extract(
                subquery,
                set(df_outcoming['items'].to_list()),
                scorer=fuzz.token_set_ratio,
                limit=1,
                # score_cutoff=75
)
[print(x) for x in answers]
if len(answers) > 0:
    print(count_common_words(subquery, answers[0][0]))
    print(count_common_digits(subquery, answers[0][0]))

pinar salam ac bitir jambon 60 gr 1 x 10
('pinar salam ac bitir jambon 60 gr 1 x 10', 100.0, 100)
1.0
1.0


In [40]:
df_outcoming[df_outcoming['items'] == '7 up 1 lt x 12']

,Unnamed: 0,#,Product or Service Name,Unit,Quantity/Volume,unit,items


In [41]:
%%time
# Обрабатываем все товары из внешних источников и находим для них пару из предобработанного стобца. Score - степень схожести
to_find = df_incoming['items'].to_list()
candidates = set(df_outcoming['items'].to_list())

res_ext, res_int, score, score_w, score_d = [], [], [], [], []
for item in tqdm(to_find[:]):
    res = proc.extract(
                item,
                candidates,
                scorer=fuzz.token_set_ratio,
                limit=1
    )[0]
    res_ext.append(item)
    res_int.append(res[0])
    score.append(res[1])
    if len(res) > 0:
        score_w.append(count_common_words(item, res[0]))
        score_d.append(count_common_digits(item, res[0]))
        

100%|██████████| 5000/5000 [00:04<00:00, 1219.41it/s]

CPU times: user 4.11 s, sys: 3.77 ms, total: 4.11 s
Wall time: 4.1 s


In [42]:
from fastparquet import ParquetFile, write as parq_write

In [43]:
df_outcoming.shape, df_incoming.shape

((5000, 7), (5000, 7))

In [44]:
comp = pd.DataFrame()

In [45]:
comp['right'] = res_ext

In [46]:
comp['left'] = res_int

In [47]:
comp['score'] = score
comp['score_w'] = score_w
comp['score_d'] = score_d

In [48]:
parq_write(
    f'{os.getcwd()}/comp.parq',
    comp,
    compression='GZIP'
)

In [49]:
comp.shape

(5000, 5)

In [50]:
comp[(comp.score > 75) & (comp.score > 0.4) & (comp.score_d > 0.4)].shape

(4753, 5)

In [51]:
comp[(comp.score > 75) & (comp.score > 0.4) & (comp.score_d > 0.4)].sample(10)

,right,left,score,score_w,score_d
1840,superfresh ton light 160 grx 2 li 320 1 x 12,superfresh ton light 160 grx 2,100.0,1.000000,0.571429
1819,bebeto starcake 35 gr 1 x 24,bebeto starcake 35 gr 1 x 24,100.0,1.000000,1.000000
185,suxari biftekli 30 gr 1 x 40,suxari biftekli 30 gr 1 x 40,100.0,1.000000,1.000000
972,suxari bavarskie kolbaski 90 gr 1 x 24,suxari bavarskie kolbaski 90 gr 1 x 24,100.0,1.000000,1.000000
3645,sevimli dad xama kaunasskaya 20 pct 200 gr 1 x 12,sevimli dad xama kaunasskaya 20 pct 200 gr 1 x 12,100.0,1.000000,1.000000
38,sevimli dad kesmik ciyelekli 0 pct 180 gr 1 x 1,sevimli dad kesmik ciyelekli 0 pct 180 gr 1 x 1,100.0,1.000000,1.000000
4634,pinar peynir labne 200 gr 1 x 12,pinar peynir labne light 200 gr 1 x 12,100.0,0.857143,1.000000
4808,sonti multivitamin 200 gr 1 x 27,sonti multivitamin 200 gr 1 x 27,100.0,1.000000,1.000000
1598,suxari biftekli 30 gr 1 x 40,suxari biftekli 30 gr 1 x 40,100.0,1.000000,1.000000
1918,suxari bavarskie kolbaski 90 gr 1 x 24,suxari bavarskie kolbaski 90 gr 1 x 24,100.0,1.000000,1.000000


In [52]:
comp.head()

,right,left,score,score_w,score_d
0,komili misir yagi 2 lt pet 1 x 9,komili misir yagi 2 lt pet 1 x 9,100.0,1.0,1.0
1,suxari sir 30 gr 1 x 40,suxari sir 30 gr 1 x 40,100.0,1.0,1.0
2,sonti multivitamin 200 gr 1 x 27,sonti multivitamin 200 gr 1 x 27,100.0,1.0,1.0
3,lays sir 50 gr 1 x 28,lays sir 50 gr 1 x 28,100.0,1.0,1.0
4,pinar salam ac bitir jambon 60 gr 1 x 10,pinar salam ac bitir jambon 60 gr 1 x 10,100.0,1.0,1.0


In [53]:
sub_result = pd.concat(
    [
        df_incoming,
        comp
    ],
    axis=1,
)

In [54]:
sub_result.head()

,Unnamed: 0,#,Product or Service Name,Unit,Quantity/Volume,unit,items,right,left,score,score_w,score_d
0,326335,326336,KOMILI MISIR YAGI 2 LT PET 1X9,ADET.LT,30,adet.lt,komili misir yagi 2 lt pet 1 x 9,komili misir yagi 2 lt pet 1 x 9,komili misir yagi 2 lt pet 1 x 9,100.0,1.0,1.0
1,18670,18671,SUXARI SIR 30 GR 1 X 40,EDED,20,eded,suxari sir 30 gr 1 x 40,suxari sir 30 gr 1 x 40,suxari sir 30 gr 1 x 40,100.0,1.0,1.0
2,230521,230522,SONTI MULTIVITAMIN 200 GR 1X27,EDED,27,eded,sonti multivitamin 200 gr 1 x 27,sonti multivitamin 200 gr 1 x 27,sonti multivitamin 200 gr 1 x 27,100.0,1.0,1.0
3,293756,293757,LAYS SIR 50 GR 1 x 28,EDED,15,eded,lays sir 50 gr 1 x 28,lays sir 50 gr 1 x 28,lays sir 50 gr 1 x 28,100.0,1.0,1.0
4,73719,73720,PINAR SALAM AC-BITIR JAMBON 60 GR 1x10,EDED,10,eded,pinar salam ac bitir jambon 60 gr 1 x 10,pinar salam ac bitir jambon 60 gr 1 x 10,pinar salam ac bitir jambon 60 gr 1 x 10,100.0,1.0,1.0


In [55]:
sub_result.shape

(5000, 12)

In [56]:
df_outcoming.head(3)

,Unnamed: 0,#,Product or Service Name,Unit,Quantity/Volume,unit,items
0,343622,343623,"BELIY QOROD SUD KOKTEYL CIYELEK 1.5% 0,2L 1X12",EDED,12,adet.lt,beliy qorod sud kokteyl ciyelek 1.5 pct 0.2 l ...
1,648980,648981,LAYS RIDGED PAPRIKA 90 GR 1 x 12,EDED,12,eded,lays ridged paprika 90 gr 1 x 12
2,424442,424443,ULKER PUDING ICIM CIYELEKLI 60 GR 1X24,EDED,24,eded,ulker puding icim ciyelekli 60 gr 1 x 24


In [57]:
df_incoming.head(3)

,Unnamed: 0,#,Product or Service Name,Unit,Quantity/Volume,unit,items
0,326335,326336,KOMILI MISIR YAGI 2 LT PET 1X9,ADET.LT,30,adet.lt,komili misir yagi 2 lt pet 1 x 9
1,18670,18671,SUXARI SIR 30 GR 1 X 40,EDED,20,eded,suxari sir 30 gr 1 x 40
2,230521,230522,SONTI MULTIVITAMIN 200 GR 1X27,EDED,27,eded,sonti multivitamin 200 gr 1 x 27


In [58]:
comp_int = sub_result.merge(df_outcoming, how='left', left_on='left', right_on='items')

In [59]:
comp_int.head()

,Unnamed: 0_x,#_x,Product or Service Name_x,Unit_x,Quantity/Volume_x,unit_x,items_x,right,left,score,score_w,score_d,Unnamed: 0_y,#_y,Product or Service Name_y,Unit_y,Quantity/Volume_y,unit_y,items_y
0,326335,326336,KOMILI MISIR YAGI 2 LT PET 1X9,ADET.LT,30,adet.lt,komili misir yagi 2 lt pet 1 x 9,komili misir yagi 2 lt pet 1 x 9,komili misir yagi 2 lt pet 1 x 9,100.0,1.0,1.0,362987,362988,KOMILI MISIR YAGI 2 LT PET 1X9,ADET.LT,24,eded,komili misir yagi 2 lt pet 1 x 9
1,326335,326336,KOMILI MISIR YAGI 2 LT PET 1X9,ADET.LT,30,adet.lt,komili misir yagi 2 lt pet 1 x 9,komili misir yagi 2 lt pet 1 x 9,komili misir yagi 2 lt pet 1 x 9,100.0,1.0,1.0,155372,155373,KOMILI MISIR YAGI 2 LT PET 1X9,ADET.LT,54,eded,komili misir yagi 2 lt pet 1 x 9
2,326335,326336,KOMILI MISIR YAGI 2 LT PET 1X9,ADET.LT,30,adet.lt,komili misir yagi 2 lt pet 1 x 9,komili misir yagi 2 lt pet 1 x 9,komili misir yagi 2 lt pet 1 x 9,100.0,1.0,1.0,545258,545259,KOMILI MISIR YAGI 2 LT PET 1X9,ADET.LT,90,eded,komili misir yagi 2 lt pet 1 x 9
3,326335,326336,KOMILI MISIR YAGI 2 LT PET 1X9,ADET.LT,30,adet.lt,komili misir yagi 2 lt pet 1 x 9,komili misir yagi 2 lt pet 1 x 9,komili misir yagi 2 lt pet 1 x 9,100.0,1.0,1.0,864077,864078,KOMILI MISIR YAGI 2 LT PET 1X9,ADET.LT,2,adet.ml,komili misir yagi 2 lt pet 1 x 9
4,18670,18671,SUXARI SIR 30 GR 1 X 40,EDED,20,eded,suxari sir 30 gr 1 x 40,suxari sir 30 gr 1 x 40,suxari sir 30 gr 1 x 40,100.0,1.0,1.0,720614,720615,SUXARI SIR 30 GR 1 X 40,EDED,10,eded,suxari sir 30 gr 1 x 40


In [60]:
comp_int.shape

(53831, 19)

In [61]:
comp_int = comp_int.drop_duplicates(subset=['#_x']) 
comp_int.shape

(5000, 19)

In [62]:
# x - incoming y - outcoming
comp_int.sample(5)

,Unnamed: 0_x,#_x,Product or Service Name_x,Unit_x,Quantity/Volume_x,unit_x,items_x,right,left,score,score_w,score_d,Unnamed: 0_y,#_y,Product or Service Name_y,Unit_y,Quantity/Volume_y,unit_y,items_y
40615,574683,574684,BERRAK JALAPENO BIBER DILIMLI 370 ML 1X12,ADET.ML,4,adet.ml,berrak jalapeno biber dilimli 370 ml 1 x 12,berrak jalapeno biber dilimli 370 ml 1 x 12,berrak biber tursusu aci 370 ml biberiye 1 x 12,75.362319,0.5,1.0,326522,326523,BERRAK BIBER TURSUSU ACI 370 ML (BIBERIYE) 1X12,ADET.ML,139,eded,berrak biber tursusu aci 370 ml biberiye 1 x 12
1380,171832,171833,DOKTOR MILK XAMA 25% 350 QR 1x12,EDED,5,eded,doktor milk xama 25 pct 350 qr 1 x 12,doktor milk xama 25 pct 350 qr 1 x 12,doktor milk xama 25 pct 350 qr 1 x 12,100.000000,1.0,1.0,353065,353066,DOKTOR MILK XAMA 25% 350 QR 1x12,EDED,3,eded,doktor milk xama 25 pct 350 qr 1 x 12
50884,83575,83576,SUPERFRESH MILFOY TEREYAGLI 1000 GR 1X10,EDED,10,eded,superfresh milfoy tereyagli 1000 gr 1 x 10,superfresh milfoy tereyagli 1000 gr 1 x 10,superfresh milfoy tereyagli 1000 gr 1 x 10,100.000000,1.0,1.0,26714,26715,SUPERFRESH MILFOY TEREYAGLI 1000 GR 1X10,EDED,10,eded,superfresh milfoy tereyagli 1000 gr 1 x 10
51717,751653,751654,CHEESELLA SACAQLI 2 KQ 1X2,EDED,2,eded,cheesella sacaqli 2 kq 1 x 2,cheesella sacaqli 2 kq 1 x 2,cheesella sacaqli 5 kq 1 x 5,96.153846,1.0,0.5,73454,73455,CHEESELLA SACAQLI 5 KQ 1X5,EDED,5,eded,cheesella sacaqli 5 kq 1 x 5
41760,478740,478741,ELVAN BARCAKE ORMAN MEYVELI 20 GR 1X24X6,EDED,24,eded,elvan barcake orman meyveli 20 gr 1 x 24 x 6,elvan barcake orman meyveli 20 gr 1 x 24 x 6,elvan barcake orman meyveli 20 gr 1 x 24 x 6,100.000000,1.0,1.0,928534,928535,ELVAN BARCAKE ORMAN MEYVELI 20 GR 1X24X6,EDED,120,eded,elvan barcake orman meyveli 20 gr 1 x 24 x 6


In [63]:
comp_int['unit_x'].value_counts()

unit_x
eded       4263
adet.lt     315
ədəd        166
adet.ml     128
adet.kg      75
adet.cl      41
litr          5
kq            3
kg            2
baql          1
amp           1
Name: count, dtype: int64

In [64]:
comp_int['unit_y'].value_counts()

unit_y
eded       4413
adet.lt     230
ədəd        198
adet.ml      67
adet.cl      46
adet.kg      31
kq           15
Name: count, dtype: int64

In [65]:
comp_int.columns

Index(['Unnamed: 0_x', '#_x', 'Product or Service Name_x', 'Unit_x',
       'Quantity/Volume_x', 'unit_x', 'items_x', 'right', 'left', 'score',
       'score_w', 'score_d', 'Unnamed: 0_y', '#_y',
       'Product or Service Name_y', 'Unit_y', 'Quantity/Volume_y', 'unit_y',
       'items_y'],
      dtype='object')

In [66]:
comp_int = comp_int[[
          'items_x', 'items_y',
          'Quantity/Volume_x', 'unit_x', 'Quantity/Volume_y', 'unit_y',
          'score', 'score_w', 'score_d',
          'Product or Service Name_x', 'Product or Service Name_y', ]]

In [67]:
comp_int_true = comp_int[(comp_int.score > 75) & (comp_int.score_w > 0.4) & (comp_int.score_d > 0.4)]

In [68]:
comp_int_doubt = comp_int.drop(comp_int_true.index, axis=0)

In [69]:
comp_int_doubt['items_y']  = ''
comp_int_doubt['Quantity/Volume_y'] = 0
comp_int_doubt['unit_y'] = ''
comp_int_doubt['Product or Service Name_y'] = ''

In [70]:
comp_scored = pd.concat([comp_int_true, comp_int_doubt])

In [71]:
comp_scored.shape

(5000, 11)

In [72]:
comp_scored.sample(10)

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score,score_w,score_d,Product or Service Name_x,Product or Service Name_y
23682,sevimli dad gatig 1.5 pct 900 gr 1 x 6,sevimli dad gatig 1.5 pct 900 gr 1 x 6,4,eded,4,eded,100.000000,1.000000,1.0,"SEVIMLI DAD GATIG 1,5% 900 GR 1X6","SEVIMLI DAD GATIG 1,5% 900 GR 1X6"
42171,sevimli dad kesmik yagli 9 pct 180 gr 1 x 1,sevimli dad kesmik yagli 9 pct 180 gr 1 x 1,6,eded,15,eded,100.000000,1.000000,1.0,SEVIMLI DAD KESMIK YAGLI 9% 180 GR 1x1,SEVIMLI DAD KESMIK YAGLI 9% 180 GR 1x1
12833,milla xama 25 pct 300 gr 1 x 24,milla xama 25 pct 300 gr 1 x 24,24,eded,24,eded,100.000000,1.000000,1.0,MILLA XAMA 25% 300GR 1X24,MILLA XAMA 25% 300GR 1X24
36115,president kesme vetcina 150 gr 1 x 15,president kesme vetcina 150 gr 1 x 15,3,eded,15,eded,100.000000,1.000000,1.0,PRESIDENT KESME VETCINA 150 GR 1X15,PRESIDENT KESME VETCINA 150 GR 1X15
30923,pinar sut kido kakaolu 180 ml 1 x 27,pinar sut kido kakaolu 180 ml 1 x 27,6,adet.ml,27,eded,100.000000,1.000000,1.0,PINAR SUT KIDO KAKAOLU 180 ML 1x27,PINAR SUT KIDO KAKAOLU 180 ML 1x27
3348,pinar sut tam yagli 1 lt 1 x 12,pinar sut tam yagli 1 lt 1 x 12,6,adet.lt,24,eded,100.000000,1.000000,1.0,PINAR SUT TAM YAGLI 1 LT 1x12,PINAR SUT TAM YAGLI 1 LT 1x12
145,pinar suzme peyniri 100 gr 1 x 12,pinar suzme peyniri 100 gr 1 x 12,5,eded,3,eded,100.000000,1.000000,1.0,PINAR SUZME PEYNIRI 100 GR 1X12,PINAR SUZME PEYNIRI 100 GR 1X12
52314,asci qirmizi lobya 800 gr 1 x 12,asci mercimek qirmizi 800 gr 1 x 12,24,eded,6,eded,89.655172,0.666667,1.0,BIZIM ASCI QIRMIZI LOBYA 800 GR 1x12,BIZIM ASCI MERCIMEK QIRMIZI 800 GR 1x12
3723,mehmet efendi kofe 100 gr 1 x 200,mehmet efendi kofe 100 gr 1 x 200,50,eded,12,eded,100.000000,1.000000,1.0,MEHMET EFENDI KOFE 100GR 1X200,MEHMET EFENDI KOFE 100GR 1X200
34729,sevimli dad kefir 2.5 pct 500 gr 1 x 30,sevimli dad kefir 2.5 pct 500 gr 1 x 30,16,eded,1,eded,100.000000,1.000000,1.0,SEVIMLI DAD KEFIR 2.5% 500 GR 1X30,SEVIMLI DAD KEFIR 2.5% 500 GR 1X30


In [73]:
comp_int.shape, comp_int_true.shape, comp_int_doubt.shape

((5000, 11), (4721, 11), (279, 11))

In [74]:
comp_int_true.sample(5)

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score,score_w,score_d,Product or Service Name_x,Product or Service Name_y
42031,sevimli dad kesmik ciyelekli 0 pct 180 gr 1 x 1,sevimli dad kesmik ciyelekli 0 pct 180 gr 1 x 1,15,eded,6,eded,100.0,1.0,1.0,SEVIMLI DAD KESMIK CIYELEKLI 0% 180 GR1x1,SEVIMLI DAD KESMIK CIYELEKLI 0% 180 GR1x1
49649,sevimli dad kefir 0.5 pct 500 gr 1 x 30,sevimli dad kefir 0.5 pct 500 gr 1 x 30,44,eded,2,eded,100.0,1.0,1.0,SEVIMLI DAD KEFIR 0.5% 500 GR 1X30,SEVIMLI DAD KEFIR 0.5% 500 GR 1X30
26176,ulker puding icim sokoladli 60 gr 1 x 24,ulker puding icim sokoladli 60 gr 1 x 24,24,eded,16,eded,100.0,1.0,1.0,ULKER PUDING ICIM SOKOLADLI 60 GR 1X24,ULKER PUDING ICIM SOKOLADLI 60 GR 1X24
10167,sevimli dad gatig 1.5 pct 900 gr 1 x 6,sevimli dad gatig 1.5 pct 900 gr 1 x 6,14,eded,4,eded,100.0,1.0,1.0,"SEVIMLI DAD GATIG 1,5% 900 GR 1X6","SEVIMLI DAD GATIG 1,5% 900 GR 1X6"
5615,superfresh superton 160 gr 1 x 24,superfresh superton 160 gr 1 x 24,2,eded,6,eded,100.0,1.0,1.0,SUPERFRESH SUPERTON 160 GR 1X24,SUPERFRESH SUPERTON 160 GR 1X24


In [75]:
import numpy as np

In [76]:
def extract_pieces_two_char(x: str):
    for pc in str(x).replace('.', ' ').split():
        if len(pc) == 2:
            return pc
    return np.nan

In [77]:
def extract_pieces_five_char(x: str):
    for pc in str(x).replace('.', ' ').split():
        if len(pc) == 4:
            return True
    return np.nan

In [78]:
comp_scored['unit_common'] = comp_scored['unit_x'] + ' ' +  comp_int_true['unit_y']

In [79]:
comp_scored['unit_common_set'] = comp_scored['unit_common'].apply(lambda x: set(str(x).replace('.', ' ').split()))

In [80]:
comp_scored['unit_2pc_x'] = comp_scored['unit_x'].apply(lambda x: extract_pieces_two_char(x))
comp_scored['unit_2pc_y'] = comp_scored['unit_y'].apply(lambda x: extract_pieces_two_char(x))

In [81]:
comp_scored['unit_common_str'] = comp_scored['unit_common_set'].apply(lambda x: str(list(x)[0]) if len(list(x)) == 0 else ' '.join(list(x)))

In [82]:
# x - incoming y - outcoming
comp_scored.sample(10)

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score,score_w,score_d,Product or Service Name_x,Product or Service Name_y,unit_common,unit_common_set,unit_2pc_x,unit_2pc_y,unit_common_str
52897,suxari mix deniz 95 gr 1 x 16,suxari mix 95 gr 1 x 16,4,eded,3,eded,100.000000,0.666667,1.000000,SUXARI MIX DENIZ 95 GR 1 X 16,SUXARI MIX ET 95 GR 1 X 16,eded eded,{eded},NaN,NaN,eded
14246,lays krab 150 gr 1 x 18,lays krab 150 gr 1 x 18,5,eded,4,eded,100.000000,1.000000,1.000000,LAYS KRAB 150 GR 1 x 18,LAYS KRAB 150 GR 1 x 18,eded eded,{eded},NaN,NaN,eded
27711,sevimli dad kesmik kaunas 0.3 pct 180 gr 1 x 1,sevimli dad kesmik kaunas 0.3 pct 180 gr 1 x 1,160,eded,20,eded,100.000000,1.000000,1.000000,SEVIMLI DAD KESMIK KAUNAS 0.3% 180 GR1x1,SEVIMLI DAD KESMIK KAUNAS 0.3% 180 GR1x1,eded eded,{eded},NaN,NaN,eded
11868,semeyniy budj qati sud 380 qr 1 x 20,semeyniy budj qati sud 380 qr 1 x 20,8,eded,15,eded,100.000000,1.000000,1.000000,SEMEYNIY BUDJ QATI SUD 380 QR 1X20,SEMEYNIY BUDJ QATI SUD 380 QR 1X20,eded eded,{eded},NaN,NaN,eded
44414,berrak tursu salatalik 720 ml 1 1 x 12,berrak tursu salatalik 720 ml 1 1 x 12,12,adet.ml,6,eded,100.000000,1.000000,1.000000,BERRAK TURSU SALATALIK 720 ML NO 1 1X12,BERRAK TURSU SALATALIK 720 ML NO 1 1X12,adet.ml eded,"{eded, adet, ml}",ml,NaN,eded adet ml
4951,sev mercimek qirmizi 700 gr 1 x 12,asci mercimek qirmizi 800 gr 1 x 12,12,eded,6,eded,89.855072,0.800000,0.666667,SEV MERCIMEK QIRMIZI 700 GR 1x12,BIZIM ASCI MERCIMEK QIRMIZI 800 GR 1x12,eded eded,{eded},NaN,NaN,eded
6503,mehmet efendi kofe 100 gr 1 x 200,mehmet efendi kofe 100 gr 1 x 200,4,eded,12,eded,100.000000,1.000000,1.000000,MEHMET EFENDI KOFE 100GR 1X200,MEHMET EFENDI KOFE 100GR 1X200,eded eded,{eded},NaN,NaN,eded
10898,bebeto cool beans fliptop sour mix 36 gr 1 x 1...,bebeto cool beans fliptop sour mix 36 gr 1 x 1...,1,eded,1,eded,100.000000,1.000000,1.000000,BEBETO COOL BEANS FLIPTOP SOUR MIX 36GR 1x12x12,BEBETO COOL BEANS FLIPTOP SOUR MIX 36GR 1x12x12,eded eded,{eded},NaN,NaN,eded
52815,berrak tursu salatalik 720 ml 1 1 x 12,berrak tursu salatalik 720 ml 1 1 x 12,6,adet.ml,6,eded,100.000000,1.000000,1.000000,BERRAK TURSU SALATALIK 720 ML NO 1 1X12,BERRAK TURSU SALATALIK 720 ML NO 1 1X12,adet.ml eded,"{eded, adet, ml}",ml,NaN,eded adet ml
40037,pinar beyaz peynir tam yagli 400 gr 1 x 12,pinar beyaz peynir tam yagli 400 gr 1 x 12,12,eded,2,eded,100.000000,1.000000,1.000000,PINAR BEYAZ PEYNIR (TAM YAGLI) 400 GR 1x12,PINAR BEYAZ PEYNIR (TAM YAGLI) 400 GR 1x12,eded eded,{eded},NaN,NaN,eded


In [83]:
comp_scored['quant_in'] = comp_scored['Quantity/Volume_x'].astype('str').str.replace(',', '.').fillna(0).astype('float')
comp_scored['quant_out'] = comp_scored['Quantity/Volume_y'].astype('str').str.replace(',', '.').fillna(0).astype('float')

In [84]:
comp_scored.groupby(by=['items_x', 'unit_common_str']).agg({'quant_in':'sum','quant_out':'sum'}).sort_values('unit_common_str')

,,quant_in,quant_out
items_x,unit_common_str,,
doktor milk kefir 0.03 pct 900 gr,adet ədəd ml,98.0,4.0
parmalat kokteyl cokolato 500 ml,adet ədəd ml,3.0,2.0
pinar kokteyli sosis 215 gr,adet ədəd ml,4.0,3.0
president dilimli maasdam 150 gr 46 pct,adet ədəd ml,4.0,10.0
fersan detox alma aromali 250 ml 1 x 24,adet ədəd ml,6.0,5.0
...,...,...,...
sevimli dad urek pendir kg,nan,93.0,0.0
ilan zəhəri əleyhinə zərdab,nan,6.0,0.0
hazer.lok.rose lemon menthe flavour 350 gr 1 x 12,nan,2.0,0.0


In [85]:
comp_scored.shape

(5000, 18)

In [86]:
def remove_numbers(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    # while text.count(2 * " ") > 0:
    #     text = text.replace(2 * " ", " ")
    return ' '.join([x for x in text.split() if len(x) > 2 and x not in ('pct', 'dad')][:3])

In [87]:
comp_scored['items_x_numberless'] = comp_scored['items_x'].apply(lambda x: remove_numbers(x))

In [88]:
comp_scored[['items_x_numberless', 'items_x']].sample(5)

,items_x_numberless,items_x
46107,pinar labne cikolatali,pinar labne cikolatali 180 gr 1 x 12
32488,bebeto moji lolli,bebeto moji lolli sakizli 16 gr 1 x 4 x 120
25139,superfresh barbunya pilaki,superfresh barbunya pilaki 400 gr 1 x 24
9942,cheetos fistikli,cheetos fistikli 50 gr 1 x 23
14075,sevimli kesmik yagsiz,sevimli dad kesmik yagsiz 0 pct 180 gr 1 x 1


In [89]:
Counter(" ".join(comp_scored['items_x_numberless']).split()).most_common(150)

[('sevimli', 828),
 ('pinar', 521),
 ('lays', 466),
 ('president', 334),
 ('kesmik', 279),
 ('xama', 236),
 ('suxari', 195),
 ('ulker', 181),
 ('bebeto', 174),
 ('labne', 158),
 ('elvan', 156),
 ('milk', 154),
 ('kefir', 153),
 ('doktor', 150),
 ('tadim', 147),
 ('peynir', 144),
 ('pendir', 142),
 ('superfresh', 138),
 ('sud', 134),
 ('gatig', 133),
 ('paprika', 132),
 ('doritos', 130),
 ('yagi', 117),
 ('kaunasskaya', 112),
 ('puding', 111),
 ('today', 105),
 ('yagli', 104),
 ('pendiri', 102),
 ('ridged', 101),
 ('qlavprodukt', 95),
 ('sir', 91),
 ('kesme', 88),
 ('klasik', 85),
 ('kaunas', 76),
 ('cheetos', 75),
 ('kirlangic', 74),
 ('sut', 74),
 ('berrak', 72),
 ('icim', 71),
 ('canax', 71),
 ('acili', 71),
 ('biftekli', 67),
 ('bagdat', 63),
 ('sochnaya', 62),
 ('icimino', 59),
 ('suse', 57),
 ('shashlik', 55),
 ('taco', 55),
 ('dolina', 54),
 ('misir', 53),
 ('sirin', 52),
 ('kere', 52),
 ('lokum', 52),
 ('ketcap', 52),
 ('salam', 50),
 ('k.m', 50),
 ('burcu', 49),
 ('zeytinyagi',

In [90]:
import re
import time
from sentence_transformers import SentenceTransformer, util

/home/varsey/PycharmProjects/drl-case/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [91]:
def clusterize(model: object, job_name_list: list, **kwargs) -> list:
    """Клсастеризация набора должностей."""
    # Apply encoding.
    print("Encode the corpus. This might take a while")
    corpus_embeddings = model.encode(
        job_name_list, 
        batch_size=32, 
        show_progress_bar=True, 
        convert_to_tensor=True
    )
    # Get clusters
    print("Start clustering")
    start_time = time.time()
    #Two parameters to tune:
    #min_cluster_size: Only consider cluster that have at least 25 elements
    #threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
    clusters = util.community_detection(corpus_embeddings, **kwargs)
    print("Clustering done after {:.2f} sec".format(time.time() - start_time))
    return clusters

In [92]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

/home/varsey/PycharmProjects/drl-case/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [93]:
len(set(comp_scored['items_x_numberless'].to_list())), len(comp_scored['items_x_numberless'].to_list())

(816, 5000)

In [94]:
job_name_list = list(set(comp_scored['items_x_numberless'].to_list()))
cluster = clusterize(model, job_name_list)

Encode the corpus. This might take a while


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

Start clustering
Clustering done after 0.13 sec


In [95]:
import random

In [96]:
for cl in cluster[:]:
    for member in random.sample(cl, 5):
        print(comp_scored['items_x_numberless'].to_list()[member])
    print(len(cl))
    print()
    

doritos taco
lays ridged pendir
doktor milk kefir
lays t?ndi̇rd?
jelaniya ketcup klassik
48

pinar salam bitir
pinar peynir labne
president kesme yagli
tadim misir soslu
elvan today bear
34

elvan panda gara
doktor milk xama
sev kend pendiri
dolcia puding bananli
suxari sir
24

president kesme chizburger
cheetos fistikli
pinar denge maden
mehmet efendi kofe
sadocok multi
20

suxari shashlik
sevimli kesmik yagli
nesquik snack latte
sevimli canax pendiri
suxari smetan
19

quickbury burger buns
doritos nacho
pinar suzme peyniri
sevimli kesmik yagli
tadim misir corn
19

bagdat tarcin kabuk
sevimli kesmik uzumlu
sevimli kesmik yagsiz
president gobelek
elvan today croissant
19

lays klasik
president sirin k.m
sandora albali
qlavprodukt adjika domasnaya
sevimli dovga
16

sevimli sud goycay
qlavprodukt xardal russkaya
elvan today aloha
lays klasik
bagdat tatli kirmizi
13

suxari smetan
sevimli dovga
president sirin k.m
sevimli kesmik kaunas
tadim antep fistigi
13

tadim findik ici
suxari bifte

In [97]:
[len(x) for x in cluster]

[48, 34, 24, 20, 19, 19, 19, 16, 13, 13, 13, 12, 12, 11, 11, 10, 10]

In [98]:
import spacy

# !python -m spacy download en_core_web_sm xx_ent_wiki_sm
nlp = spacy.load("en_core_web_sm")
for member in random.sample(cluster[:1][0], 10):
    doc = nlp(comp_scored['items_x_numberless'].to_list()[member])
    
    for tok in doc:
        print(tok, tok.pos_)
    print()

bebeto PROPN
spaghetti PROPN
apple NOUN

doktor PROPN
milk NOUN
kefir NOUN

doritos VERB
nacho NOUN

lays VERB
klasik PROPN

sevimli PROPN
xama PROPN
kaunasskaya NOUN

pinar PROPN
peynir PROPN
taze NOUN

doritos VERB
taco NOUN

sevimli PROPN
kesmik PROPN
ciyelekli PROPN

president PROPN
kesme PROPN
yagli PROPN

milla PROPN
ayran PROPN
naneli PROPN



In [99]:
import yake

In [100]:
kw_extractor = yake.KeywordExtractor(
    n=2,
    dedupLim=0.5,
    dedupFunc='sequencematcher',
    windowsSize=2,
    top=3,
    features=None
)

for job in comp_scored.sample(50)['items_x_numberless']:
    keywords = kw_extractor.extract_keywords(job)
    print(job)
    for kw in keywords:
        print(kw)
    print('--')
    x = {kw[0]: kw[1] for kw in keywords}
    # print({k: v for k, v in sorted(x.items(), key=lambda item: item[1])})
    print(min(x, key=x.get))
    fltr = kw_extractor.extract_keywords((min(x, key=x.get)))
    fltr_dct = {kw[0]: kw[1] for kw in fltr}
    print(max(fltr_dct, key=fltr_dct.get))
    print()

icimino ciyelek
('icimino ciyelek', 0.02570861714399338)
('icimino', 0.15831692877998726)
('ciyelek', 0.15831692877998726)
--
icimino ciyelek
icimino

sevimli canax pendiri
('sevimli canax', 0.04940384002065631)
('canax pendiri', 0.04940384002065631)
('canax', 0.29736558256021506)
--
sevimli canax
canax

euro.stanich.. blaqoyar hdy
('blaqoyar hdy', 0.2766850298235749)
('hdy', 0.46553351027698087)
--
blaqoyar hdy
hdy

doritos taco
('doritos taco', 0.02570861714399338)
('taco', 0.15831692877998726)
--
doritos taco
taco

berrak tursu karisik
('berrak tursu', 0.04940384002065631)
('tursu karisik', 0.04940384002065631)
('berrak', 0.15831692877998726)
--
berrak tursu
berrak

lays ridged xardal
('lays ridged', 0.04940384002065631)
('ridged xardal', 0.04940384002065631)
('lays', 0.15831692877998726)
--
lays ridged
lays

sochnaya dolina multivitamin
('sochnaya dolina', 0.04940384002065631)
('dolina multivitamin', 0.04940384002065631)
('dolina', 0.29736558256021506)
--
sochnaya dolina
dolina

ba

In [101]:
def get_2main_word(word: str):
    keywords = kw_extractor.extract_keywords(word)
    x = {kw[0]: kw[1] for kw in keywords}
    if len(x) > 0:
        return min(x, key=x.get)
    else:
        return ''

def get_1main_word(words: str):
    if len(words) > 0:
        fltr = kw_extractor.extract_keywords(words)
        fltr_dct = {kw[0]: kw[1] for kw in fltr}
        return max(fltr_dct, key=fltr_dct.get)
    else:
        return ''

In [102]:
for item in comp_scored.sample(50)['items_x_numberless']:
    words = get_2main_word(item)
    word = get_1main_word(words)
    print(words)
    print(word)
    print()

sevimli canax
canax

sevimli qaymaq
sevimli

ulker pendir
ulker

kirlangic misir
misir

doritos acili
acili

pinar sut
sut

superfresh kozl.krmz
superfresh

elvan today
elvan

sevimli kesmik
sevimli

tadim findik
tadim

berrak sarimsak
berrak

doritos taco
taco

quickbury buns
buns

pinar sut
sut

elvan today
elvan

hugo reitzel
hugo

lays paprika
lays

berrak tursu
berrak

suxari steyk
suxari

elvan today
elvan

zolotoy petusok
zolotoy

pinar suzme
pinar

sevimli kefir
kefir

doritos taco
taco

delverde farfallex
delverde

berrak tursu
berrak

superfresh ton
ton

pinar suzme
pinar

ulker puding
ulker

kirlangic aycicek
aycicek

lays krab
lays

cheesella top
top

sevimli xama
xama

bagdat pul
pul

doktor milk
milk

pinar peynir
pinar

sevimli qaymaq
sevimli

sev manni
sev

pinar krema
pinar

sevimli dovga
dovga

lays paprika
lays

sandora portagal
sandora

qlavprodukt badimcan
badimcan

karsa cikobest
karsa

sevimli kefir
kefir

sevimli canax
canax

lays baharat
lays

fersan elma
elma


In [103]:
comp_scored['main_2word'] = comp_scored['items_x_numberless'].apply(lambda x: get_2main_word(x))
comp_scored['main_1word'] = comp_scored['main_2word'].apply(lambda x: get_1main_word(x))

In [104]:
comp_scored[['main_1word', 'main_2word', 'items_x_numberless', 'items_x']].sample(10)

,main_1word,main_2word,items_x_numberless,items_x
47271,sevimli,sevimli kaunass,sevimli kaunass kere,sevimli dad kaunass kere yagi 1 kg 82.5 pct 1 ...
12516,sizma,kirlangic sizma,kirlangic sizma zeytinyagi,kirlangic sizma zeytinyagi 2 lt pet 1 x 8
9752,milk,doktor milk,doktor milk gatig,doktor milk gatig suse 0.5 pct 500 gr 1 x 12
32622,gatig,sevimli gatig,sevimli gatig,sevimli dad gatig 1.5 pct 900 gr 1 x 6
49649,kefir,sevimli kefir,sevimli kefir,sevimli dad kefir 0.5 pct 500 gr 1 x 30
4492,sirin,president sirin,president sirin k.m,president sirin k.m dolce erik 20 pct 200 gr 1...
44963,turist,turist klassik,turist klassik,turist klassik 100 gr 1 x 30
3039,gatig,sevimli gatig,sevimli gatig suse,sevimli dad gatig suse 0.5 pct 500 gr 1 x 12
14345,superfresh,superfresh kozl.krmz,superfresh kozl.krmz biber,superfresh kozl.krmz biber konservesi 4.2 kg 1...
39684,bebeto,bebeto sutlumisir,bebeto sutlumisir,bebeto 8 gr sutlumisir 1 kg 1 x 12


In [105]:
comp_scored.groupby(by=['main_2word', 'items_x_numberless', 'Product or Service Name_x', 'Product or Service Name_y', 'unit_common_str']).agg({'quant_in':'sum','quant_out':'sum'}).sort_values('unit_common_str')

,,,,,quant_in,quant_out
main_2word,items_x_numberless,Product or Service Name_x,Product or Service Name_y,unit_common_str,,
pinar kokteyli,pinar kokteyli sosis,PINAR KOKTEYLI SOSIS 215 GR,PINAR SOSIS KOKTEYL 215 GR 1x18,adet ədəd ml,4.0,3.0
president dilimli,president dilimli maasdam,PRESIDENT DILIMLI MAASDAM 150 GR 46%,PRESIDENT DILIMLI MAASDAM 150 GR 46% 1X10,adet ədəd ml,4.0,10.0
parmalat kokteyl,parmalat kokteyl cokolato,PARMALAT KOKTEYL COKOLATO 500 ML,PARMALAT KOKTEYL COKOLATO 500 ML,adet ədəd ml,3.0,2.0
fersan detox,fersan detox alma,FERSAN DETOX ALMA AROMALI 250 ML 1X24,FERSAN DETOX LIMON AROMALI 250 ML 1X24,adet ədəd ml,6.0,5.0
doktor milk,doktor milk kefir,"DOKTOR MILK KEFIR 0,03 % 900 GR","DOKTOR MILK KEFIR 0,03 % 900 QR 1x12",adet ədəd ml,98.0,4.0
...,...,...,...,...,...,...
pinar truva,pinar truva kunefe,PINAR TRUVA KUNEFE 125 GR 1x12,,nan,18.0,0.0
superfresh ton,superfresh ton aycicek,SUPERFRESH TON AYCICEK YAG. 80 GR 3 LU,,nan,12.0,0.0
lays duzlu,lays duzlu cipsi,LAYS DUZLU CIPSI 150 GR,,nan,306.0,0.0


In [106]:
import lda
import json
import logging
import collections
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


class StageLDA:
    def __init__(self, texts_data: pd.DataFrame, topics_number: int = 7):
        self.texts_data = texts_data
        self.topics_number = topics_number

    def run(self) -> str:
        anons_lda = collections.defaultdict(list)
        text_lda = collections.defaultdict(list)
        model = lda.LDA(n_topics=self.topics_number, random_state=123)
        logging.getLogger("lda").setLevel(logging.ERROR)

        keys = list(self.texts_data.index)
        corpus = self.texts_data['items_x_numberless'].to_list()

        vec = CountVectorizer(analyzer='word', ngram_range=(1, 3))
        x = vec.fit_transform(corpus)

        model.fit(x)
        doc_topic = model.transform(x)

        for i, memberships in enumerate(doc_topic):
            indices = np.argsort(memberships)[::-1]
            anons_lda[int(indices[0])].append(keys[i])
            text_lda[int(indices[0])].append(corpus[i])

        res = json.dumps(text_lda, indent=1, ensure_ascii=False)

        return json.loads(res)


In [107]:
def get_analyzed_texts(texts_data: pd.DataFrame):
    topics_number = 25 # int(texts_data.shape[0] / 100)
    stage = StageLDA(texts_data=texts_data, topics_number=topics_number)
    res = stage.run()

    for x in range(topics_number):
        print(x)
        [print(x) for x in set(res.get(str(x)))]


In [108]:
get_analyzed_texts(comp_scored.sample(1000))

0
ulker icimino mese
suxari smetan
sandora multi
sevimli dovga
bagdat kimyon
lays stix ketchup
suxari sir
sokovita multi sire
xrus team suxari
tavanic
icimino sud ciyelekli
icimino sud bananli
suxari kalmar
dolcia puding bananli
sadocok multi
icimino sud sokoladli
mimi sud bananli
beliy qorod sud
lays stix cheddar
tadim favori mixed
1
hugo reitzel tursu
elvan today goldies
president emmental pendiri
elvan today aloha
elvan today snowball
elvan today cloud
elvan today sufle
elvan today donut
hugo reitzel xiyar
elvan barcake orman
lays rebrishki qril
elvan today roll
elvan panda gara
elvan today bear
elvan today croissant
elvan today bites
2
berlinki shokoladli konfet
sandora portagal
dari kubani multivitamin
xlorami̇n
frico old dutch
pinar peynir bulgarevo
suxari shashlik
piyale galeta unu
sandora shaftali
suxari bavarskie kolbaski
angry birds shaftali
didi shaftali aromali
bagdan feyxoa kompotu
dari kubani portagal
lays shashlik
bagdat galeta unu
dari kubani alma
angry birds alma
3
pin

In [109]:
df_incoming[df_incoming['Product or Service Name'] == 'SEVIMLI DAD KESMIK GAYSILI 9% 180 GR 1x1']['unit'].unique()

array(['eded'], dtype=object)

In [110]:
df_outcoming[df_outcoming['Product or Service Name'] == 'SEVIMLI DAD KESMIK ALBALILI 0% 180 GR1x1']['unit'].unique()

array(['adet.lt', 'eded', 'adet.cl', 'ədəd'], dtype=object)

In [111]:
import pickle

In [112]:
with open('res_int.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(res_int, outp, pickle.HIGHEST_PROTOCOL)

In [113]:
with open('res_ext.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(res_ext, outp, pickle.HIGHEST_PROTOCOL)

In [114]:
with open('score.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(score, outp, pickle.HIGHEST_PROTOCOL)